#Boson sampling

# Background

Boson sampling is a simplified model of quantum computation that uses non‐interacting bosons—typically photons—passing through a linear optical network to sample from a probability distribution that is believed to be hard to simulate classically.

## Basic Setup

1. **Input State:**  
   We begin by preparing an input state with $ n $ single photons distributed over $ m $ modes. A common choice is
   $$
   |\psi_{\text{in}}\rangle = |1,1,\dots,1,0,0,\dots,0\rangle,
   $$
   where the first $ n $ modes each contain one photon and the remaining $ m-n $ modes are in the vacuum state.

2. **Linear Optical Interferometer:**  
   The interferometer is described by an $ m \times m $ unitary matrix $ U $. It transforms the creation operators at the input as
   $$
   \hat{b}_j^\dagger = \sum_{i=1}^m U_{ji}\,\hat{a}_i^\dagger,
   $$
   where $\hat{a}_i^\dagger , \hat{b}_j^\dagger$ are the creation operators for the input and output modes, respectively.

3. **Output State:**  
   After the transformation, the state becomes
   $$
   |\psi_{\text{out}}\rangle = \hat{U}|\psi_{\text{in}}\rangle.
   $$

4. **Measurement:**  
   When we measure the photon numbers at the output (i.e., perform Fock state measurements), we obtain an output configuration
   $$
   S = (s_1, s_2, \dots, s_m),
   $$
   with $\sum_{j=1}^m s_j = n.$

## Sampling Probabilities

The probability \( p(S) \) of detecting the configuration $ S $ is given by
$$
p(S) = \frac{\left|\operatorname{Per}(U_S)\right|^2}{s_1!\,s_2!\,\cdots\,s_m!},
$$
where:
- $U_S $ is an $n \times n$ submatrix of $ U $, formed by selecting the columns corresponding to the input modes containing photons and the rows corresponding to the output modes where photons are detected.
- $ \operatorname{Per}(U_S) $ is the **permanent** of $ U_S $.

## The Permanent

The permanent of an $ n \times n $ matrix $ A $ is defined as
$$
\operatorname{Per}(A) = \sum_{\sigma \in S_n} \prod_{i=1}^n A_{i,\sigma(i)},
$$
where $ S_n $ is the set of all permutations of $ \{1, 2, \dots, n\} $. Unlike the determinant, the permanent does not include any sign factors and is known to be a $\#P$-hard problem, which implies that even approximating it is computationally difficult for classical computers.

## Why Is Boson Sampling Interesting?

- **Computational Hardness:**  
  Since calculating the permanent is $\#P$-hard, sampling from the boson sampling output distribution is believed to be infeasible for classical computers as $ n $ grows.

- **Quantum Advantage:**  
  Although boson sampling does not constitute a universal quantum computer, its ability to perform a task that is hard for classical machines makes it a promising candidate for demonstrating quantum supremacy.


In [1]:
%%capture
!pip install strawberryfields
!pip install thewalrus

In [2]:
import numpy as np
from thewalrus import perm
# set the random seed
np.random.seed(42)


In [3]:
from scipy.stats import unitary_group

# define the linear interferometer
U = unitary_group.rvs(4)
print(U)

[[ 0.23648826-0.48221431j  0.06829648+0.04447898j  0.51150074-0.09529866j
   0.55205719-0.35974699j]
 [-0.11148167+0.69780321j -0.24943828+0.08410701j  0.46705929-0.43192981j
   0.16220654-0.01817602j]
 [-0.22351926-0.25918352j  0.24364996-0.05375623j -0.09259829-0.53810588j
   0.27267708+0.66941977j]
 [ 0.11519953-0.28596729j -0.90164923-0.22099186j -0.09627758-0.13105595j
  -0.0200152 +0.12766128j]]


In [4]:
# import Strawberry Fields
import strawberryfields as sf
from strawberryfields.ops import *

# initialize a 4 mode program
boson_sampling = sf.Program(4)

with boson_sampling.context as q:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac     | q[2]
    Fock(1) | q[3]
    Interferometer(U) | q

In [5]:
boson_sampling.compile(compiler="fock").print()

Fock(1) | (q[0])
Fock(1) | (q[1])
Vac | (q[2])
Fock(1) | (q[3])
Rgate(1.713) | (q[0])
BSgate(0.3206, 0) | (q[0], q[1])
Rgate(-2.748) | (q[2])
BSgate(0.9024, 0) | (q[2], q[3])
Rgate(-2.461) | (q[1])
BSgate(1.456, 0) | (q[1], q[2])
Rgate(0.9034) | (q[0])
BSgate(1.007, 0) | (q[0], q[1])
Rgate(2.564) | (q[0])
Rgate(4.645) | (q[1])
Rgate(2.701) | (q[2])
Rgate(0.9826) | (q[3])
BSgate(-1.395, 0) | (q[2], q[3])
Rgate(0.1149) | (q[2])
BSgate(-0.4808, 0) | (q[1], q[2])
Rgate(0.7131) | (q[1])


In [6]:
# run the engine
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 7})
results = eng.run(boson_sampling)

In [7]:
# extract the joint Fock probabilities
probs = results.state.all_fock_probs()

#Comparing to the permanent

The permanent of a matrix, defined by

$$
\operatorname{Per}(A)=\sum_{\sigma=S_N} \prod_{i=1}^N A_{i \sigma(i)}
$$

where $S_N$ is the set of all permutations of $N$ elements, is calculated in a similar fashion to the determinant, but unlike the determinant, the signatures of the permutations are not taken into account - every permutation is taken as a positive quantity.

In graph theory, the permanent calculates the number of perfect matchings in a bipartite graph with adjacency matrix $A$.

  $$
  p(S) = \frac{|\operatorname{Per}(U_S)|^2}{s_1!\,s_2!\,\cdots\,s_m!}.
  $$

In [8]:
BS_theory = np.abs(perm(U[:, [0,1,3]][[0,1,3]]))**2 / 1
SF_experiment =  probs[1,1,0,1]
print("Experimental :",SF_experiment)
print("Theororical :", BS_theory)
print("Exact percentage difference :",100*np.abs(BS_theory-SF_experiment)/BS_theory)

Experimental : 0.11854373036507135
Theororical : 0.11854373036507143
Exact percentage difference : 7.024135868717443e-14


## Summary

In boson sampling:
- **Input:** \( n \) single photons in \( m \) modes.
- **Transformation:** The photons pass through a linear interferometer described by a unitary \( U \).
- **Measurement:** The output configuration \( S \) is measured in the Fock basis.
- **Probability:** The measurement probability involves the permanent of a submatrix of \( U \),
  $$
  p(S) = \frac{|\operatorname{Per}(U_S)|^2}{s_1!\,s_2!\,\cdots\,s_m!}.
  $$
- **Implication:** Since the permanent is hard to compute classically, boson sampling is seen as evidence of quantum computational advantage.